In [1]:
# Import libraries
# !pip install yfinance
# !pip install fredapi

import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
from fredapi import Fred
# from bs4 import BeautifulSoup
from datetime import datetime
import json 
import requests as re
pd.set_option('display.max_columns', None)


In [2]:
# Function to calculate CAPM
def calculate_capm(stock_ticker, market_ticker, risk_free_rate, start_date, end_date):
    # Fetch historical data
    stock_data = yf.download(stock_ticker, start=start_date, end=end_date)
    market_data = yf.download(market_ticker, start=start_date, end=end_date)

    # Calculate daily returns
    stock_returns = stock_data['Adj Close'].pct_change().dropna()
    market_returns = market_data['Adj Close'].pct_change().dropna()

    # Align the returns
    returns = pd.concat([stock_returns, market_returns], axis=1)
    returns.columns = ['Stock', 'Market']
    returns = returns.dropna()

    # Calculate beta using regression
    X = sm.add_constant(returns['Market'])  # Add constant for intercept
    model = sm.OLS(returns['Stock'], X).fit()
    beta = model.params[1]  # Beta is the slope of the regression

    # Calculate expected return using CAPM formula
    expected_return = risk_free_rate + beta * (returns['Market'].mean() * 252 - risk_free_rate)

    return beta, expected_return


In [3]:
# Get API from the FED

fred = Fred(api_key = '87f6fd22bd418ae0d91a107ad639d347')
ten_year_treasury_rate = fred.get_series_latest_release('GS10') / 100

# Set the risk-free rate

risk_free_rate = ten_year_treasury_rate.iloc[-1]

# print(ten_year_treasury_rate)
print(risk_free_rate)

0.0387


# Actual Calculation

In [4]:
"""
# Get the S&P 500 tickers from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
snp500_table = pd.read_html(url)

# The S&P 500 tickers are in the first table on the Wikipedia page
snp500_df = snp500_table[0]

# Extract the tickers
stock_tickers = snp500_df['Symbol'].tolist()

# Display the tickers
print(stock_tickers)
capm_df = pd.DataFrame()

"""

'\n# Get the S&P 500 tickers from Wikipedia\nurl = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"\nsnp500_table = pd.read_html(url)\n\n# The S&P 500 tickers are in the first table on the Wikipedia page\nsnp500_df = snp500_table[0]\n\n# Extract the tickers\nstock_tickers = snp500_df[\'Symbol\'].tolist()\n\n# Display the tickers\nprint(stock_tickers)\ncapm_df = pd.DataFrame()\n\n'

In [5]:
stock_tickers = [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "GOOGL", # Alphabet Inc.
    "NVDA",  # NVIDIA Corporation
    "JNJ",   # Johnson & Johnson
    "PFE",   # Pfizer Inc.
    "JPM",   # JPMorgan Chase & Co.
    "XOM",   # Exxon Mobil Corporation
    "NEE",   # NextEra Energy Inc.
    "AMZN",  # Amazon.com, Inc.
    "TSLA",  # Tesla, Inc.
    "CAT",   # Caterpillar Inc.
    "BA",    # Boeing Company
    "KO",    # Coca-Cola Company
    "PG",    # Procter & Gamble Co.
    "VTI",   # Vanguard Total Stock Market ETF
    "VOO",   # Vanguard S&P 500 ETF
    "QQQ",    # Invesco QQQ Trust
    "FTNT",
    "^GSPC",
    "SOFI"
]

# stock_tickers = [
#         "FTNT"
# ]


capm_df = pd.DataFrame()
# display(capm_df)

In [6]:
market_ticker = '^GSPC'  # S&P 500 Index
start_date = '2019-01-01'
end_date = datetime.today()

betas = []
expected_returns = []
stock_lists = []

for stock_ticker in stock_tickers:
    beta, expected_return = calculate_capm(stock_ticker, market_ticker, risk_free_rate, start_date, end_date)
    stock_lists.append(stock_ticker)
    betas.append(beta)
    expected_returns.append(expected_return)

capm_df['stock_ticker'] = stock_lists
capm_df['beta'] = betas
capm_df['expected_returns'] = expected_returns

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\namph\AppData\Local\Temp\ipykernel_19864\1153167210.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = model.params[1]  # Beta is the slope of the regression
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\namph\AppData\Local\Temp\ipykernel_19864\1153167210.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = model.params[1]  # Beta is the slope of the regress

In [7]:
capm_df = capm_df.sort_values(by = 'expected_returns', ascending=False)
# Display all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
display(capm_df[:100])

,stock_ticker,beta,expected_returns
3,NVDA,1.793734,0.265732
10,TSLA,1.537411,0.233289
20,SOFI,1.991505,0.224005
12,BA,1.432797,0.220048
0,AAPL,1.205690,0.191303
18,FTNT,1.196200,0.190102
1,MSFT,1.181654,0.188261
2,GOOGL,1.142157,0.183262
17,QQQ,1.122271,0.180745
9,AMZN,1.088253,0.176439
